In [0]:
%run /Workspace/Users/vishal.krishnan@neudesic.com/Bronze_to_Silver/utils

In [0]:
##  Web Analytics 
df_web = spark.read.format("delta").load(bronze_base + "Web_analytics").drop("_rescued_data")
df_web = clean_common(df_web).dropDuplicates(["session_id"])
date_format = "M/d/yyyy"


df_web = df_web.select(
    col("session_id"),
    col("campaign_id"),
    col("campaign_name"),
    to_date(col("campaign_start_date"), date_format).alias("campaign_start_date"),
    to_date(col("campaign_end_date"), date_format).alias("campaign_end_date"),
    to_timestamp("session_start").alias("session_start"),
    to_timestamp("session_end").alias("session_end"),
    col("bounce").cast(BooleanType()).alias("bounce"),
    col("signed_up").cast(BooleanType()).alias("signed_up"),
    col("purchased").cast(BooleanType()).alias("purchased"),
    col("revenue").cast(DoubleType()).alias("revenue"),
    col("user_id"),
    col("age").cast(IntegerType()).alias("age"),
    col("gender"),
    col("region"),
    current_timestamp().alias("ingestion_timestamp"),
    lit(current_user).alias("modified_by")
)

df_web = df_web.dropna(how="any")
display(df_web)
df_web.write.mode("overwrite").format("delta").save(silver_base + "Web_analytics")